In [36]:
# Local imports
from utils import file_exists, evaluate_model, eval_env_random_actions
from icecream import ic
import pandas as pd
import numpy as np

from plotnine import *

%matplotlib inline

In [2]:
# frozen-lake-ex1.py
import gym # loading the Gym library
 
# env = gym.make("FrozenLake-v1")
env = gym.make("FrozenLake8x8-v1")

env.reset()                    
env.render()



SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [28]:
print(env.action_space)
print(env.observation_space)

Discrete(4)
Discrete(64)


()

In [32]:
random_action = env.action_space.sample() # Do random actions
print(random_action)

1


In [62]:
# state, reward, done, info = 


  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


# Initialization

In [48]:
env_name = 'FrozenLake-v1'

# # Create folder to save models
# directory_path = 'models'
# Path(directory_path).mkdir(parents=True, exist_ok=True)

# num_steps = 100_000
# model_file_name = Path(directory_path, env_name+'_'+str(num_steps))

env = gym.make(env_name,
#  is_slippery=False,map_name="8x8"
 )

In [49]:
random_action = env.action_space.sample() # Do random actions # LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3

state = env.reset()
state, reward, done, info = env.step(random_action)
ic(random_action)
ic(state, reward, done, info) 
env.render()

ic| random_action: 0
ic| state: 4
    reward: 0.0
    done: False
    info: {'prob': 0.3333333333333333}


  (Left)
SFFF
FHFH
FFFH
HFFG


## Testing random actions

In [62]:
# eval_env_random_actions(env, episodes=2)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [63]:
env.render(mode="rgb_array")

  (Left)
SFFF
FHFH
FFFH
HFFG


KeyboardInterrupt: 

In [64]:
a = env.render(mode='ansi')
type(a)
a.split('\n')

['  (Left)', 'SFFF', '\x1bF\x1bHFH', 'FFFH', 'HFFG', '']

In [ ]:
print(env.action_space)
print(env.observation_space)

## Create model

In [ ]:
model = PPO(MlpPolicy, env, verbose=0)

# Training

In [66]:

def value_iteration(env, max_iterations=100000, lmbda=0.9):
  stateValue = [0 for i in range(env.nS)]
  newStateValue = stateValue.copy()
  for i in range(max_iterations):
    for state in range(env.nS):
      action_values = []      
      for action in range(env.nA):
        state_value = 0
        for i in range(len(env.P[state][action])):
          prob, next_state, reward, done = env.P[state][action][i]
          state_action_value = prob * (reward + lmbda*stateValue[next_state])
          state_value += state_action_value
        action_values.append(state_value)      #the value of each action
        best_action = np.argmax(np.asarray(action_values))   # choose the action which gives the maximum value
        newStateValue[state] = action_values[best_action]  #update the value of the state
    if i > 1000: 
      if sum(stateValue) - sum(newStateValue) < 1e-04:   # if there is negligible difference break the loop
        break
        print(i)
    else:
      stateValue = newStateValue.copy()
  return stateValue 

def get_policy(env,stateValue, lmbda=0.9):
  policy = [0 for i in range(env.nS)]
  for state in range(env.nS):
    action_values = []
    for action in range(env.nA):
      action_value = 0
      for i in range(len(env.P[state][action])):
        prob, next_state, r, _ = env.P[state][action][i]
        action_value += prob * (r + lmbda * stateValue[next_state])
      action_values.append(action_value)
    best_action = np.argmax(np.asarray(action_values))
    policy[state] = best_action
  return policy 


def get_score(env, policy, episodes=1000):
  misses = 0
  steps_list = []
  for episode in range(episodes):
    observation = env.reset()
    steps=0
    while True:
      
      action = policy[observation]
      observation, reward, done, _ = env.step(action)
      steps+=1
      if done and reward == 1:
        # print('You have got the fucking Frisbee after {} steps'.format(steps))
        steps_list.append(steps)
        break
      elif done and reward == 0:
        # print("You fell in a hole!")
        misses += 1
        break
  print('----------------------------------------------')
  print('You took an average of {:.0f} steps to get the frisbee'.format(np.mean(steps_list)))
  print('And you fell in the hole {:.2f} % of the times'.format((misses/episodes) * 100))
  print('----------------------------------------------')



env = gym.make('FrozenLake-v1')

stateValues = value_iteration(env, max_iterations=100000)
policy = get_policy(env, stateValues)
get_score(env, policy,episodes=1000)

----------------------------------------------
You took an average of 37 steps to get the frisbee
And you fell in the hole 28.00 % of the times
----------------------------------------------


# Evaluation

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay loop controls style="height: 400px;">
        <source src="data:video/mp4;base64,{}" type="video/mp4" />
        </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id)])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env, video_folder=video_folder,
        record_video_trigger=lambda step: step == 0, video_length=video_length,
        name_prefix=prefix)
        
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
    eval_env.close()

## Visualize trained agent

In [ ]:
record_video(env_name, model, video_length=500, prefix='ppo2-'+env_name)

In [ ]:
show_videos('videos', prefix='ppo2')